In [1]:
import pandas as pd
import itertools, nltk, string 
import requests, re
from nltk import Tree
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import os
import spacy
from gensim.models.wrappers import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, roc_curve
import matplotlib.pyplot as plt
from math import exp, expm1, log, log10
from scipy.spatial.distance import euclidean
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.distance import cdist

import ast



from sklearn.metrics import average_precision_score,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score
from collections import OrderedDict
import numpy as np
import math
import statistics
from pywsd.lesk import simple_lesk
from nltk.corpus import sentiwordnet as swn
from nltk.parse.corenlp import CoreNLPDependencyParser
parser = CoreNLPDependencyParser()



wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")

Warming up PyWSD (takes ~10 secs)... took 17.34176516532898 secs.


In [2]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))

read_lexicon()

In [101]:
#load function

linking_verbs_be = [
    'be',
    'is',
    'are',
    'am',
    'was',
    'were',
    'can be',
    'could be',
    'will be',
    'would be',
    'shall be',
    'should be',
    'may be',
    'might be',
    'must be',
    'has been',
    'have been',
    'had been'
];

linking_verbs_v = [
    'feel',
    'look',
    'smell',
    'sound',
    'taste',
    'act',
    'appear',
    'become',
    'get',
    'grow',
    'prove',
    'remain', 
    'seem',
    'stay',
    'turn'
];

def check_is_noun(pos):
    return re.match('NN.*', pos)

def check_is_prp(pos):
    return re.match('PRP.*', pos)

def check_is_verb(pos):
    return re.match('VB.*', pos)

def check_is_adjective(pos):
    return re.match('JJ.*', pos)

def check_is_adverb(pos):
    return re.match('RB.*', pos)

def lemmatize(word, pos):
    tag = None#wn.NOUN
    if(check_is_noun(pos)):
        tag = wn.NOUN
    elif(check_is_verb(pos)):
        tag = wn.VERB
    elif(check_is_adjective(pos)):
        tag = wn.ADJ
    elif(check_is_adverb(pos)):
        tag = wn.ADV
    if tag:        
        lemma = wordnet_lemmatizer.lemmatize(word, tag)
    else:
        lemma = word
    return lemma


def preprocessing(sentence):
    res = sentence.replace("'m", "am").replace("n't", " not").replace("dont", "do not").replace("’s", ' is').replace("'s", ' is').replace("'ve", " have").lower()
    res = re.sub(r'\b\d+\b', 'NUM', res)
    res = re.sub(r'(.)\1{3,}', '', res)
    to_delete_punc = set(string.punctuation) - {',', '.', '(', ')', '-'} # remove comma and fullstop
    clean_tokens = [x for x in nltk.word_tokenize(res) if x not in to_delete_punc]

    return " ".join(clean_tokens)

def preprocessing_with_lemma(sentence):
    res = re.sub(r'[^\w\s]',' ', sentence.replace("'m", "am").replace("n't", "not").replace("dont", "do not").replace("'ve", " have").replace("'s", ' is').replace("’s", ' is')).lower()
    res = re.sub(r'(.)\1{3,}', '', res)
   
    #checking for parallel clauses
    #splitted = res.split(', and but')
    res = re.sub(r'\b\d+\b', '', res)
    tagged_words = pos_tag(res.encode())
    lemma = []
    for word, pos in tagged_words:
        if word not in stopWords:
            lemma.append(lemmatize(word, pos)) 
        
    return " ".join(lemma)
    
def pos_tag(sentence):
    #url = "http://localhost:9000"
    #request_params = {"annotators": "pos"}
    #r = requests.post(url, data=sentence.encode('utf-8'), params=request_params, timeout=120)
    #try:
    #    results = r.json()['sentences'][0]['tokens']
    #    res = []
    #    for pos in results:
    #        res.append((pos['word'], pos['pos']))
    #    return res
    #except Exception as e:
    #    print(e)
    #    return []
    
    return nltk.pos_tag(nltk.word_tokenize(sentence))
    
def get_tregex(text, tregex):
    url = "http://localhost:9000/tregex"
    request_params = {"pattern": tregex}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    pattern = r'(?<= )[a-zA-Z,.].*?(?=\))'
    replaced = s.replace('\r\n', '')
    res = ' '.join(re.findall(pattern, replaced))
    return res
        
def sentence_type(clauses):
    IC = 0
    DC = 0
    for clause in clauses:
        if(clause[1] == 'IC'):
            IC += 1
        elif(clause[1] == 'DC'):
            DC += 1

    if IC == 1 and DC == 0:
        return 'simple_sentence'
    elif IC >= 2 and DC == 0:
        return 'compound_sentence'
    elif IC ==1 and DC >= 1:
        return 'complex_sentence'
    elif IC > 1 and DC >= 1:
        return 'compound_complex_sentence'
    else:
        return 'phrase'
    

In [4]:
def geometric_medianX(points, method='auto', options={}):
    """
    Calculates the geometric median of an array of points.
    method specifies which algorithm to use:
        * 'auto' -- uses a heuristic to pick an algorithm
        * 'minimize' -- scipy.optimize the sum of distances
        * 'weiszfeld' -- Weiszfeld's algorithm
    """

    points = np.asarray(points)

    if len(points.shape) == 1:
        # geometric_median((0, 0)) has too much potential for error.
        # Did the user intend a single 2D point or two scalars?
        # Use np.median if you meant the latter.
        raise ValueError("Expected 2D array")

    if method == 'auto':
        if points.shape[1] > 2:
            # weiszfeld tends to converge faster in higher dimensions
            method = 'weiszfeld'
        else:
            method = 'minimize'

    return _methods[method](points, options)


def minimize_method(points, options={}):
    """
    Geometric median as a convex optimization problem.
    """

    # objective function
    def aggregate_distance(x):
        return cdist([x], points).sum()

    # initial guess: centroid
    centroid = points.mean(axis=0)

    optimize_result = minimize(aggregate_distance, centroid, method='COBYLA')

    return optimize_result.x


def weiszfeld_method(points, options={}):
    """
    Weiszfeld's algorithm as described on Wikipedia.
    """

    default_options = {'maxiter': 1000, 'tol': 1e-7}
    default_options.update(options)
    options = default_options

    def distance_func(x):
        return cdist([x], points)

    # initial guess: centroid
    guess = points.mean(axis=0)

    iters = 0

    while iters < options['maxiter']:
        distances = distance_func(guess).T

        # catch divide by zero
        # TODO: Wikipedia cites how to deal with distance 0
        distances = np.where(distances == 0, 1, distances)

        guess_next = (points/distances).sum(axis=0) / (1./distances).sum(axis=0)

        guess_movement = np.sqrt(((guess - guess_next)**2).sum())

        guess = guess_next

        if guess_movement <= options['tol']:
            break

        iters += 1

    return guess


_methods = {
    'minimize': minimize_method,
    'weiszfeld': weiszfeld_method,
}

In [18]:
def graph(radii, theta, key, show_pic=False):
    i=0
    a=[]
    b=[]
    rad=list(radii.values())
    theta1=list(theta.values())
    index = []
    while(i<len(radii)):
        if(theta1[i] != 0):
            a.append(rad[i]*math.cos(math.radians(theta1[i])))
            b.append(rad[i]*math.sin(math.radians(theta1[i])))
            index.append(i)
        i+=1 
        
    if show_pic:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot([0,0],[-1,1])
        plt.plot([-1,1],[0,0])
        plt.axis([-1,1,-1,1])

        plt.scatter(a,b,label="circles",color="r",marker="o",s=10)
        q = 0
        rad2=list(radii.keys())
        for i,j,z in zip(a,b, index):
            ax.annotate('%s' %rad2[z], xy=(i,j), xytext=(15,0), textcoords='offset points')
            #ax.annotate('%s' %'', xy=(i,j), xytext=(15,0), textcoords='offset points')
            
            q = q+1    
        
        points = []
        for x,y in zip(a,b):
            points.append([x, y])
        
        if(len(points) == 0):
            points.append([0, 0])
            
        senti_x, senti_y = geometric_medianX(points)
        plt.scatter(senti_x,senti_y,label="circles",color="b",marker="o",s=10)
        ax.annotate('%s' % '', xy=(senti_x,senti_y), xytext=(15,0), textcoords='offset points')
           
        ax.add_artist(plt.Circle((0,0),1.0,color='b',fill=False))
        plt.xlabel('Sentiment Strength')
        plt.ylabel('Orientation')
        plt.title(key)
        plt.savefig("graph.png")
    
    return a,b

In [19]:
negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "but",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection",
  #  'even',
  #  'although'
]


def analyse_file(key, lines, lemma_lines, usingSynonym=False, usingCoocurrence=False):    
    radii, synonym_keys, coocurrence_keys = get_TDOC(lines, key, lemma_lines, usingSynonym, usingCoocurrence)    
    return radii, synonym_keys, coocurrence_keys

def get_TDOC(lines, key, lemma_lines, usingSynonym=False, usingCoocurrence=False, synonym_keys={}, coocurrence_keys={}):
    freq = {'Init': 0}              #Number of times context term occurs with key
    freq.clear()
    prohib = ['even', 'main'] #stopWords
    for line in lemma_lines:
        words = line.split(" ")
        tagged = pos_tag(line)
        if key in words:
            if usingSynonym:
                syn_c=[]
                for context, pos in tagged:
                    try:
                        synonym_keys.setdefault(context, [])
                                    
                        for i, syn in enumerate(wn.synsets(context)):
                            if i > 4:
                                break
                            
                            if(syn.pos() != wn.NOUN):
                                for l in syn.lemmas():
                                    if not '-' in l.name():
                                        if(l.name() not in synonym_keys[context]):
                                            synonym_keys[context].append(l.name())
                                        syn_c.append(l.name())

                    except:
                        #print(context)
                        #print('error')
                        pass
                for c in syn_c:
                    if c not in words:
                        words.append(c)
            for context in words:
                flag=0
                for i in prohib:
                    if i == context:
                        flag=1
                        break
                if flag==0 and context!=key:# and context in op_set:
                    freq.setdefault(context, 0)
                    freq[context] = freq.get(context) + 1
                                           
    N = 0                           #Total Number of terms in Document
    for line in lemma_lines:
        words = line.split(" ")
        N += len(words)
    if usingCoocurrence:
        di = {}
        coocurrence_keys.setdefault(key, [])
        it = 0
        for key_n in freq.keys():
            tg = pos_tag(key_n)
            #if it > 5:
            #    break;
            if( check_is_adjective(tg[0][1]) or (key_n in positive_lexicon or key_n in negative_lexicon)):
                for line in lemma_lines:
                    words = line.split(" ")
                    tagged = pos_tag(line)
                    
                    if key_n in words:
                        for x in range(0, len(words)):
                            try:
                                #if (words[x+1] == key_n) or (words[x+2] == key_n) or (words[x+3] == key_n) or (words[x-1] == key_n):
                                if check_is_adjective(tagged[x][1]):
                                    di.setdefault(words[x], 0)
                                    di[words[x]] = di.get(words[x]) + 1
                                    if words[x] not in coocurrence_keys[key]:
                                        coocurrence_keys[key].append(words[x])
                            except:
                                pass
               
        freq.update(di)
        
    Nci = {'Init': 0}               #Total terms that occur with context term
    Nci.clear()
    for context in freq.keys():
        for line in lemma_lines:
            words = line.split(" ")
            if context in words:
                Nci.setdefault(context, 0)
                Nci[context] += len(words)
                
    if usingSynonym:
        for context in freq.keys():
            if context not in Nci.keys():
                Nci.setdefault(context, 1)

    radii = {'Init': 0}             #Get Radius of context term with TDOC formula
    radii.clear()
    df = pd.DataFrame(columns=['c', 'm', 'N', 'Nc', 'f', 'N/Nc', 'log(N/Nc)', 'fxlog(N/Nc)', '/4'])
    max_value = 0
    for term in freq.keys():
        radii[term] = (freq[term]*(log(N/Nci[term])))
        
        if radii[term] > max_value:
            max_value = radii[term]
    for term in freq.keys():
        radii[term] = radii[term]/max_value
        
        df = df.append({'c': term,
                'm': key,
                'N': N,
                'Nc': Nci[term],
                'f': freq[term],
                'N/Nc': "{0:.2f}".format(N/Nci[term]),
                'log(N/Nc)': "{0:.2f}".format(log(N/Nci[term])),
                'fxlog(N/Nc)': "{0:.2f}".format(freq[term]*(log(N/Nci[term]))),
                'normalisasi': "{0:.2f}".format((freq[term]*(log(N/Nci[term])))/max_value)
               }, ignore_index=True)
    
    #df.to_excel("tdoc2.xlsx")
    return radii, synonym_keys, coocurrence_keys                  #Returns entire set of context terms related to key

def get_theta(key, sentences):
    scores = []
    for sentence in sentences:
        flag = True
        
        pp_tagged = pos_tag(sentence)
        tagged = ('','')
        for p in pp_tagged:
            if p[0] == key:
                tagged = p
        if tagged == ('', ''):
            flag = False
        
        ambiguous = tagged[0]
        tag = tagged[1]
        pos = ''

        if 'NN' in tag or 'NNS' in tag:
            pos = 'n'
        elif 'VB' in tag:
            pos = 'v'
        elif 'VBG' in tag:
            pos = 'v'
        elif 'JJ' in tag:
            pos = 'a'
        elif 'RB' in tag:
            pos = 'r'
        else:
            flag = False

        if flag:
            if ambiguous in negation or pos == 'n' or pos == 'v' or pos == 'r':
                scores.append(0)
                continue
             
            try:
                 answer = simple_lesk(sentence, ambiguous, pos)
            except Exception as e:
                print(e)
                answer = None
                
            if answer:
                score = swn.senti_synset(answer.name())
                endscore = 0
                if score.pos_score() > score.neg_score():
                    endscore = score.pos_score()
                elif score.neg_score() > score.pos_score():
                    endscore = score.neg_score() * (-1)
                else:
                    endscore = 0
                
                words = sentence.split(' ')
                word_around = []
                for x in range(0, len(words)):
                    if words[x] in string.punctuation:
                        continue
                    try:
                        if (words[x+1] == key) or (words[x+2] == key) or (words[x+3] == key) or (words[x+4] == key):
                            word_around.append(words[x])
                        elif (words[x-1] == key):
                            word_around.append(words[x])
                    except:
                        pass
                for neg in negation:
                    if neg in word_around:
                       # print('masuk', endscore, key, score)
                        endscore *= (-1)
                        break
                        
                scores.append(endscore)
            else:
                scores.append(0)
        else:
            scores.append(0)
    final_score = np.average(scores)
    
    return np.pi * final_score

def prior_sentiment(radii, key, all_sentences, sentence_lemmas, usingSynonym=False, usingCoocurrence=False, synonym_keys={}, coocurrence_keys={}):
    theta = {'Init': 0}
    theta.clear()
    #print('masuk prior', synonym_keys, usingSynonym)
    for word in radii.keys():
        sentences = []
        for sentence, sentence_lemma in zip(all_sentences, sentence_lemmas):
            words = sentence_lemma.split(' ')
            if key in words:
                
                if usingSynonym:
                    child_syns = synonym_keys[key]
                    
                    if word in child_syns:
                        words.append(word)
                        sentence = sentence.replace(key, word)
                        
                   
                if (word in words):
                    sentences.append(sentence)
                            
            if usingCoocurrence:
             
                if word in words and word != key:
                   # print(sentence, k)
                    if sentence not in sentences:
                        sentences.append(sentence)
            #      
            #            print(sentences, k, key)
            #            sentences.append(sentence)
                        
        
        #print("+++", sentences, "+++")
        if(len(sentences) > 0):
           # print(word, sentences)
            filter = get_theta(word, sentences)            #if function returns 0 word does not exist in lexicon
            #print('hore')
        else:
            #print('kok bisa', word)
            filter = 0.0
        theta[word] = filter
        #print('---', theta[word], '---')
        
    return theta

In [20]:
def senti(keys, lines, lemma_lines, usingSynonym=False, usingCoocurrence=False):
    points_k = []
    for key in keys:
        if key in memoize:
            radii = memoize[key]['radii']
            theta = memoize[key]['theta']
        else:
            radii, synonym_keys, coocurrence_keys = analyse_file(key, lines, lemma_lines, usingSynonym, usingCoocurrence)
            #print(synonym_keys)
            theta = prior_sentiment(radii, key, lines, lemma_lines, usingSynonym, usingCoocurrence, synonym_keys, coocurrence_keys)
            memoize.setdefault(key, {})
                   
            memoize[key]['radii'] = radii
            memoize[key]['theta'] = theta
            
        a,b = graph(radii, theta, key)
        points = []
        for x,y in zip(a,b):
            points.append([x, y])
        
        if(len(points) == 0):
            points.append([0, 0])
        
        senti_x, senti_y = geometric_medianX(points)
        
        points_k.append([senti_x, senti_y])
    
    sentiment_strength, sentiment_orientation = geometric_medianX(points_k)
    #print(sentiment_strength, sentiment_orientation)
    if sentiment_orientation > 0:
        return 'positive', sentiment_orientation
    else:
        return 'negative', sentiment_orientation

In [8]:
def categorize_dataset(category, dataset):
    return dataset[dataset.apply(lambda x: set([category]).issubset(ast.literal_eval(x['entity_prediction'])), axis=1)].reset_index(drop=True)

In [27]:
def confusion_metrix_sentiment_senticircle(selected, dataset):
    tp = 0
    tn = 0
    fp = 0
    fn = 0 
    
    for i, predictions in enumerate(dataset['opinion_prediction']):
        target = dataset['Polarity'][i]
        
        entity_predictions =  ast.literal_eval(dataset['entity_prediction'][i])
        #print(entity_predictions, predictions)
        #print(entity_predictions)
        entities = dataset['Kategori'][i].split(',')
        pred_selected = None
        actual_selected = None
        
        for index, pred in enumerate(predictions):
            if entity_predictions[index].lower() == selected.lower():
                if pred == 'positive':
                    pred_selected = True
                else:
                    pred_selected = False
        
        for index, ent in enumerate(target.split(',')):
            if entities[index].lower() == selected.lower():
                if ent == 'positive':
                    actual_selected = True
                else:
                    actual_selected = False
        if pred_selected is None or actual_selected is None:
            continue
            
        #print(pred_selected, actual_selected, entity_predictions, entities, target, predictions)
        if pred_selected and actual_selected:
            tp +=1
        elif pred_selected and not actual_selected:   
            fp += 1
        elif not pred_selected and actual_selected:
            fn += 1
        elif not pred_selected and not actual_selected:
            tn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return tp, tn, fp, fn, precision, recall, ((precision * recall) / (precision + recall)) * 2

In [10]:
data_ac3 = pd.read_csv('ac3_tripadvisor.csv')

In [11]:
data_food = categorize_dataset('FOOD', data_ac3)

In [12]:
data_price = categorize_dataset('PRICE', data_ac3)

In [13]:
data_ambience = categorize_dataset('AMBIENCE', data_ac3)

In [14]:
data_service = categorize_dataset('SERVICE', data_ac3)

SA1

In [37]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_food['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_food['preprocessed_sentence'].tolist(), data_food['preprocessed_sentence_lemma'].tolist())
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)
    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_food['opinion_prediction'] = op_pred
data_food['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [34]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_ambience['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_ambience['preprocessed_sentence'].tolist(), data_ambience['preprocessed_sentence_lemma'].tolist())
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_ambience['opinion_prediction'] = op_pred
data_ambience['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [32]:
op_pred = []
memoize = {}
op_score = []


for i, opinion_sents in enumerate(data_service['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_service['preprocessed_sentence'].tolist(), data_service['preprocessed_sentence_lemma'].tolist())
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)


    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_service['opinion_prediction'] = op_pred
data_service['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [30]:
op_pred = []
memoize = {}
op_score = []


for i, opinion_sents in enumerate(data_price['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_price['preprocessed_sentence'].tolist(), data_price['preprocessed_sentence_lemma'].tolist())
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)


    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_price['opinion_prediction'] = op_pred
data_price['senti_score'] = op_score

In [31]:
sentiment_price = (confusion_metrix_sentiment_senticircle('PRICE', data_price)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_price)

0.5


In [33]:
sentiment_service = (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_service)

0.8712871287128713


In [36]:
sentiment_ambience = (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_ambience)

0.9454545454545454


In [38]:
sentiment_food = (confusion_metrix_sentiment_senticircle('FOOD', data_food)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_food)

0.8695652173913043


In [39]:
(sentiment_price + sentiment_ambience + sentiment_food + sentiment_service) / 4

0.7965767228896802

SA2

In [40]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_food['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score =  senti(opinions, data_food['preprocessed_sentence'].tolist(), data_food['preprocessed_sentence_lemma'].tolist(), True)
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_food['opinion_prediction'] = op_pred
data_food['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [41]:
op_pred = []
op_score = []

memoize = {}
for i, opinion_sents in enumerate(data_ambience['opinion_term_prediction']):
    temp = []
    temp1 = []
    
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_ambience['preprocessed_sentence'].tolist(), data_ambience['preprocessed_sentence_lemma'].tolist(), True)
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_ambience['opinion_prediction'] = op_pred
data_ambience['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [42]:
op_pred = []
op_score = []

memoize = {}
for i, opinion_sents in enumerate(data_price['opinion_term_prediction']):
    temp = []
    temp1 = []
    
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_price['preprocessed_sentence'].tolist(), data_price['preprocessed_sentence_lemma'].tolist(), True)
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)
    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_price['opinion_prediction'] = op_pred
data_price['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [43]:
op_pred = []
op_score = []


memoize = {}
for i, opinion_sents in enumerate(data_service['opinion_term_prediction']):
    temp = []
    temp1 = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_service['preprocessed_sentence'].tolist(), data_service['preprocessed_sentence_lemma'].tolist(), True)
        temp.append(polarity)
        temp1.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp1)
    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_service['opinion_prediction'] = op_pred
data_service['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [44]:
sentiment_price = (confusion_metrix_sentiment_senticircle('PRICE', data_price)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_price)

0.7142857142857143


In [45]:
sentiment_ambience = (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[6])# + confusion_metrix_sentiment_senticircle('negative', data_ambience)[6] ) / 2
print(sentiment_ambience)

0.9357798165137614


In [46]:
sentiment_food = (confusion_metrix_sentiment_senticircle('FOOD', data_food)[6])# + confusion_metrix_sentiment_senticircle('negative', data_food)[6] ) / 2
print(sentiment_food)

0.8828125


In [47]:
sentiment_service = (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[6])# + confusion_metrix_sentiment_senticircle('negative', data_service)[6] ) / 2
print(sentiment_service)

0.8979591836734694


In [49]:
(sentiment_price + sentiment_ambience + sentiment_food + sentiment_service) / 4

0.8577093036182363

SA3

In [ ]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_food['opinion_term_prediction']):
    temp = []
    temp_score = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_food['preprocessed_sentence'].tolist(), data_food['preprocessed_sentence_lemma'].tolist(), True, True)
        temp.append(polarity)
        temp_score.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp_score)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_food['opinion_prediction'] = op_pred
data_food['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [103]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_service['opinion_term_prediction']):
    temp = []
    temp_score = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_service['preprocessed_sentence'].tolist(), data_service['preprocessed_sentence_lemma'].tolist(), True, True)
        temp.append(polarity)
        temp_score.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp_score)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_service['opinion_prediction'] = op_pred
data_service['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [104]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_ambience['opinion_term_prediction']):
    temp = []
    temp_score = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_ambience['preprocessed_sentence'].tolist(), data_ambience['preprocessed_sentence_lemma'].tolist(), True, True)
        temp.append(polarity)
        temp_score.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp_score)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_ambience['opinion_prediction'] = op_pred
data_ambience['senti_score'] = op_score

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

KeyboardInterrupt: 

In [ ]:
op_pred = []
op_score = []
memoize = {}
for i, opinion_sents in enumerate(data_price['opinion_term_prediction']):
    temp = []
    temp_score = []
    opinion_sents = ast.literal_eval(opinion_sents)
    for opinions in opinion_sents:
        polarity, senti_score = senti(opinions, data_price['preprocessed_sentence'].tolist(), data_price['preprocessed_sentence_lemma'].tolist(), True, True)
        temp.append(polarity)
        temp_score.append(senti_score)
    op_pred.append(temp)
    op_score.append(temp_score)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_price['opinion_prediction'] = op_pred
data_price['senti_score'] = op_score

In [ ]:
sentiment_price = (confusion_metrix_sentiment_senticircle('PRICE', data_price)[6]) #+ confusion_metrix_sentiment_senticircle('negative', data_value)[6] ) / 2
print(sentiment_price)

In [ ]:
sentiment_ambience = (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[6])# + confusion_metrix_sentiment_senticircle('negative', data_ambience)[6] ) / 2
print(sentiment_ambience)

In [ ]:
sentiment_food = (confusion_metrix_sentiment_senticircle('FOOD', data_food)[6])# + confusion_metrix_sentiment_senticircle('negative', data_food)[6] ) / 2
print(sentiment_food)

In [ ]:
sentiment_service = (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[6])# + confusion_metrix_sentiment_senticircle('negative', data_service)[6] ) / 2
print(sentiment_service)

In [ ]:
(sentiment_price + sentiment_ambience + sentiment_food + sentiment_service) / 4

In [ ]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[4]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[4]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[4]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[4])) / 4

In [ ]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[5]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[5]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[5]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[5])) / 4

In [ ]:
data_food.to_csv('SA3_food_tripadvisor.csv')
data_ambience.to_csv('SA3_ambience_tripadvisor.csv')
data_price.to_csv('SA3_price_tripadvisor.csv')
data_service.to_csv('SA3_service_tripadvisor.csv')

sentiwordnet

In [90]:
def predict_sentiwordnet(opinions, sent):
    endscore = 0
    for opinion in opinions:
        try:
            score = swn.senti_synset(opinion + '.a.1')
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)
            
            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
        except:
            pass
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

In [91]:
op_pred = []
for i, opinion_sents in enumerate(data_food['opinion_term_prediction']):
    temp = []
    review = data_food['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_food['opinion_prediction'] = op_pred

In [92]:
op_pred = []
for i, opinion_sents in enumerate(data_service['opinion_term_prediction']):
    temp = []
    review = data_service['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_service['opinion_prediction'] = op_pred

In [93]:
op_pred = []
for i, opinion_sents in enumerate(data_ambience['opinion_term_prediction']):
    temp = []
    review = data_ambience['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_ambience['opinion_prediction'] = op_pred

In [94]:
op_pred = []
for i, opinion_sents in enumerate(data_price['opinion_term_prediction']):
    temp = []
    review = data_price['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_price['opinion_prediction'] = op_pred

In [95]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[4]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[4]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[4]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[4])) / 4

0.8546349434082576

In [96]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[5]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[5]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[5]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[5])) / 4

0.7046889966475096

In [97]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[6]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[6]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[6]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[6])) / 4

0.7701639255278727

SentiWordnet + WSD

In [82]:
def predict_sentiwordnet_lesk(opinions, sent):
    endscore = 0
    for opinion in opinions:
        try:
            op_with_tag = pos_tag(opinion)[0]
            if check_is_noun(op_with_tag[1]):
                score = swn.senti_synset(simple_lesk(sent, opinion, pos='n').name())
            elif check_is_adjective(op_with_tag[1]):
                score = swn.senti_synset(simple_lesk(sent, opinion, pos='a').name())
            elif check_is_verb(op_with_tag[1]):
                score = swn.senti_synset(simple_lesk(sent, opinion, pos='v').name())
            elif check_is_adverb(op_with_tag[1]):
                score = swn.senti_synset(simple_lesk(sent, opinion, pos='r').name())
            else:
                score = swn.senti_synset(simple_lesk(sent, opinion, pos='n').name())
                
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)
                
            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
           
        except Exception as e:
            #print(e)
            #print(opinion, sent, op_with_tag)
            pass
            
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

In [83]:
op_pred = []
for i, opinion_sents in enumerate(data_food['opinion_term_prediction']):
    temp = []
    review = data_food['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet_lesk(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_food['opinion_prediction'] = op_pred

In [84]:
op_pred = []
for i, opinion_sents in enumerate(data_service['opinion_term_prediction']):
    temp = []
    review = data_service['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet_lesk(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_service['opinion_prediction'] = op_pred

In [85]:
op_pred = []
for i, opinion_sents in enumerate(data_ambience['opinion_term_prediction']):
    temp = []
    review = data_ambience['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet_lesk(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_ambience['opinion_prediction'] = op_pred

In [86]:
op_pred = []
for i, opinion_sents in enumerate(data_price['opinion_term_prediction']):
    temp = []
    review = data_price['Review'][i]
    opinion_sents = ast.literal_eval(opinion_sents)
    
    for opinions in opinion_sents:
        temp.append(predict_sentiwordnet_lesk(opinions, review))
    op_pred.append(temp)

    #    op_pred.append(prediction_entity(sim_combined, category))
    #y_pred.append(aspect_predictions)

data_price['opinion_prediction'] = op_pred

In [87]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[4]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[4]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[4]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[4])) / 4

0.80324263502455

In [88]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[5]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[5]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[5]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[5])) / 4

0.6801364942528736

In [89]:
((confusion_metrix_sentiment_senticircle('PRICE', data_price)[6]) + (confusion_metrix_sentiment_senticircle('FOOD', data_food)[6]) + (confusion_metrix_sentiment_senticircle('AMBIENCE', data_ambience)[6]) + (confusion_metrix_sentiment_senticircle('SERVICE', data_service)[6])) / 4

0.7343189224441821

## perhitungan aspect bases sentiment analisis

In [ ]:
data_sa3_food = pd.read_csv('SA3_food_tripadvisor.csv')
data_sa3_ambience = pd.read_csv('SA3_ambience_tripadvisor.csv')
data_sa3_service = pd.read_csv('SA3_service_tripadvisor.csv')
data_sa3_price = pd.read_csv('SA3_price_tripadvisor.csv')

food

In [ ]:
total_pos = 0
total_neg = 0

for i, predictions in enumerate(data_sa3_food['opinion_prediction']):
    entity_predictions =  ast.literal_eval(data_sa3_food['entity_prediction'][i])
    entities = data_sa3_food['entity'][i].split(',')
    predictions = ast.literal_eval(predictions)
    for index, pred in enumerate(predictions):
       # print(pred)
        if entity_predictions[index] == 'FOOD':
            if pred == 'positive':
                total_pos += 1
            else:
                total_neg += 1

print(total_pos, total_neg)  